In [5]:
#Open the data and convert the data into something the NN can read
#echo 1 | sudo tee /proc/sys/vm/overcommit_memory
#export LD_LIBRARY_PATH=${LD_LIBRARY_PATH}:/usr/local/cuda/lib64

import numpy as np

Path = "Quarter32Bit"

AllData = np.load(Path + "/ImageDataNorm.npy")
AllLabel = np.load(Path + "/ImageLab.npy")

No_Crystals = AllData.shape[0]
Total_ints = AllData.shape[0] * AllData.shape[1] * AllData.shape[2] * AllData.shape[3] 

Shuffle_Index = np.arange(No_Crystals)
np.random.shuffle(Shuffle_Index)

###########
TrainRatio = 0.9
ValidationRatio = 0.1 ## A fraction of the Traning data

###########

TrainCrystalNo = round(TrainRatio * No_Crystals)
ValCrystalNo = round(ValidationRatio * No_Crystals)
PixleNo = AllData.shape[2] * AllData.shape[3]

train_images = np.zeros(TrainCrystalNo * PixleNo * AllData.shape[1], dtype = np.float32).reshape(TrainCrystalNo, AllData.shape[1], AllData.shape[2], AllData.shape[3])
val_images = np.zeros(ValCrystalNo * PixleNo * AllData.shape[1], dtype = np.float32).reshape(ValCrystalNo, AllData.shape[1], AllData.shape[2], AllData.shape[3])

train_lab = np.zeros(TrainCrystalNo * AllData.shape[1], dtype = np.uint8).reshape(TrainCrystalNo, AllData.shape[1])
val_lab = np.zeros(ValCrystalNo * AllData.shape[1], dtype = np.uint8).reshape(ValCrystalNo, AllData.shape[1])

for i in range(0, No_Crystals): #Put Shuffled Crystals into training and validation and test
	if(i < TrainCrystalNo):
		train_images[i] = AllData[Shuffle_Index[i]]
		train_lab[i] = AllLabel[Shuffle_Index[i]]

	else:
		val_images[i - TrainCrystalNo] = AllData[Shuffle_Index[i]]
		val_lab[i - TrainCrystalNo] = AllLabel[Shuffle_Index[i]]

Shuffle_Index_Train = np.arange(TrainCrystalNo * AllData.shape[1])
Shuffle_Index_Val = np.arange(ValCrystalNo * AllData.shape[1])

np.random.shuffle(Shuffle_Index_Train)
np.random.shuffle(Shuffle_Index_Val)

train_images = train_images.reshape(-1, AllData.shape[2], AllData.shape[3], 1)
val_images = val_images.reshape(-1, AllData.shape[2], AllData.shape[3], 1)
train_lab = train_lab.reshape(-1)
val_lab = val_lab.reshape(-1)

train_images_copy = train_images
val_images_copy = val_images
train_lab_copy = train_lab
val_lab_copy = val_lab


for i in range(0,TrainCrystalNo * AllData.shape[1]):
	train_images_copy[i] = train_images[Shuffle_Index_Train[i]]
	train_lab_copy[i] = train_lab[Shuffle_Index_Train[i]]
for i in range(0,ValCrystalNo * AllData.shape[1]):
	val_images_copy[i] = val_images[Shuffle_Index_Val[i]]
	val_lab_copy[i] = val_lab[Shuffle_Index_Val[i]]

train_images = train_images_copy.astype("float32")
val_images = val_images_copy.astype("float32")
train_lab = train_lab_copy
val_lab = val_lab_copy

from keras.utils import to_categorical
train_lab = to_categorical(train_lab)
val_lab = to_categorical(val_lab)

####################################################################################

In [6]:
#Print out the shapes of the image and label arrays
print("train_images",train_images.shape)
print("val_images",val_images.shape)

print("train_lab",train_lab.shape)
print("val_lab",val_lab.shape)

train_images (58570, 64, 64, 1)
val_images (6510, 64, 64, 1)
train_lab (58570, 10)
val_lab (6510, 10)


In [26]:
#Create a convolutional NN and show the model
from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image_dataset_from_directory
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import l2
#from keras.models import load_model
import os
import matplotlib.pyplot as plt

os.environ["TF_CPP_MIN_LOG_LEVEL"]="3" 

def create_model():
    model = models.Sequential()
    model.add(layers.SeparableConv2D(128, (3, 3), activation='relu', input_shape=(64, 64, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.SeparableConv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.SeparableConv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.25))
    model.add(layers.Dense(512, activation='relu', kernel_regularizer = l2(0.0001)))
    model.add(layers.Dense(10, activation='softmax', kernel_regularizer = l2(0.0001)))
    return(model)

#model.summary()

In [34]:
RatioToTest = np.array([0.0001, 0.005, 0.01, 0.05, 0.1, 0.2, 0.4, 0.6, 0.8, 0.9, 1.0])
NumberToTest = list(map(int, (RatioToTest * TrainCrystalNo * AllData.shape[1])))
ValAccuracy = np.zeros(len(NumberToTest), dtype = np.float32)
print(NumberToTest)
for i in range(0, len(NumberToTest)):
    EarlyStop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
    model = create_model()
    model.compile(loss='categorical_crossentropy', optimizer="rmsprop", metrics=['acc'])
    
    history = model.fit(train_images[0:NumberToTest[i]], train_lab[0:NumberToTest[i]], epochs=50, batch_size = 32, validation_data = (val_images, val_lab), shuffle = True, callbacks=[EarlyStop])
    
    test_loss, test_acc = model.evaluate(val_images, val_lab) #Warning this is validation, take care if copying this code

    val_acc = history.history['val_acc']
    ValAccuracy[i] = val_acc[-1]


print(ValAccuracy)

[5, 292, 585, 2928, 5857, 11714, 23428, 35142, 46856, 52713, 58570]
Epoch 1/50
1/1 [==============================] - 1s 1s/step - loss: 2.3968 - acc: 0.0000e+00 - val_loss: 2.3793 - val_acc: 0.1014
Epoch 2/50
1/1 [==============================] - 1s 1s/step - loss: 2.3072 - acc: 0.6000 - val_loss: 2.4067 - val_acc: 0.1014
Epoch 3/50
1/1 [==============================] - 1s 1s/step - loss: 1.9672 - acc: 0.6000 - val_loss: 2.7176 - val_acc: 0.1014
Epoch 4/50
1/1 [==============================] - 1s 1s/step - loss: 1.4600 - acc: 0.6000 - val_loss: 3.6670 - val_acc: 0.1014
Epoch 5/50
1/1 [==============================] - 1s 1s/step - loss: 1.1572 - acc: 0.6000 - val_loss: 4.0710 - val_acc: 0.1014
Epoch 6/50
1/1 [==============================] - 1s 1s/step - loss: 1.0836 - acc: 0.6000 - val_loss: 4.4911 - val_acc: 0.1014
Epoch 7/50
1/1 [==============================] - 1s 1s/step - loss: 1.0352 - acc: 0.6000 - val_loss: 5.0763 - val_acc: 0.1014
Epoch 8/50
1/1 [=======================

19/19 [==============================] - 1s 78ms/step - loss: 1.8204 - acc: 0.3009 - val_loss: 1.8381 - val_acc: 0.2857
Epoch 23/50
19/19 [==============================] - 1s 76ms/step - loss: 1.7948 - acc: 0.3077 - val_loss: 1.9554 - val_acc: 0.2628
Epoch 24/50
19/19 [==============================] - 1s 77ms/step - loss: 1.7910 - acc: 0.3299 - val_loss: 1.8523 - val_acc: 0.2800
Epoch 25/50
19/19 [==============================] - 2s 80ms/step - loss: 1.7800 - acc: 0.3094 - val_loss: 1.8605 - val_acc: 0.2879
Epoch 26/50
19/19 [==============================] - 1s 78ms/step - loss: 1.8029 - acc: 0.3060 - val_loss: 1.8836 - val_acc: 0.2820
Epoch 27/50
19/19 [==============================] - 1s 73ms/step - loss: 1.7624 - acc: 0.3385 - val_loss: 1.9320 - val_acc: 0.2805
Epoch 28/50
19/19 [==============================] - 2s 81ms/step - loss: 1.7464 - acc: 0.3470 - val_loss: 1.9200 - val_acc: 0.2607
Epoch 29/50
19/19 [==============================] - 1s 79ms/step - loss: 1.7658 - acc: 

204/204 [==============================] - 1s 6ms/step - loss: 2.1961 - acc: 0.4518
Epoch 1/50
367/367 [==============================] - 7s 19ms/step - loss: 2.0094 - acc: 0.2290 - val_loss: 1.8696 - val_acc: 0.2759
Epoch 2/50
367/367 [==============================] - 7s 18ms/step - loss: 1.8528 - acc: 0.2826 - val_loss: 2.1888 - val_acc: 0.2366
Epoch 3/50
367/367 [==============================] - 7s 18ms/step - loss: 1.7600 - acc: 0.3206 - val_loss: 1.7671 - val_acc: 0.3344
Epoch 4/50
367/367 [==============================] - 7s 18ms/step - loss: 1.6927 - acc: 0.3477 - val_loss: 1.7324 - val_acc: 0.3487
Epoch 5/50
367/367 [==============================] - 7s 18ms/step - loss: 1.6442 - acc: 0.3780 - val_loss: 1.6974 - val_acc: 0.3478
Epoch 6/50
367/367 [==============================] - 7s 18ms/step - loss: 1.5908 - acc: 0.3980 - val_loss: 1.7863 - val_acc: 0.3143
Epoch 7/50
367/367 [==============================] - 7s 18ms/step - loss: 1.5438 - acc: 0.4279 - val_loss: 1.6775 - v

1465/1465 [==============================] - 23s 15ms/step - loss: 1.2701 - acc: 0.5830 - val_loss: 1.4939 - val_acc: 0.5178
Epoch 6/50
1465/1465 [==============================] - 23s 15ms/step - loss: 1.1978 - acc: 0.6234 - val_loss: 1.5361 - val_acc: 0.5490
Epoch 7/50
1465/1465 [==============================] - 22s 15ms/step - loss: 1.1375 - acc: 0.6572 - val_loss: 1.4890 - val_acc: 0.5551
Epoch 8/50
1465/1465 [==============================] - 23s 15ms/step - loss: 1.0924 - acc: 0.6856 - val_loss: 1.4867 - val_acc: 0.5614
Epoch 9/50
1465/1465 [==============================] - 23s 16ms/step - loss: 1.0600 - acc: 0.7047 - val_loss: 1.5766 - val_acc: 0.5774
Epoch 10/50
1465/1465 [==============================] - 23s 16ms/step - loss: 1.0338 - acc: 0.7198 - val_loss: 1.5871 - val_acc: 0.5705
Epoch 11/50
1465/1465 [==============================] - 23s 16ms/step - loss: 1.0087 - acc: 0.7321 - val_loss: 1.5755 - val_acc: 0.5768
Epoch 12/50
1465/1465 [==============================] - 

[292, 585, 2928, 5857, 11714, 23428, 35142, 46856, 52713, 58570]
 9/10 [==========================>...] - ETA: 0s - loss: 1.7084 - acc: 0.3611
Epoch 00001: val_loss improved from 1.77758 to 1.67347, saving model to BestModCov2.hdf5
204/204 [==============================] - 1s 6ms/step - loss: 1.6735 - acc: 0.3668
HELLO 0.36301368474960327
17/19 [=========================>....] - ETA: 0s - loss: 1.5848 - acc: 0.3952
Epoch 00001: val_loss did not improve from 1.67347
204/204 [==============================] - 1s 6ms/step - loss: 1.7774 - acc: 0.3516
HELLO 0.3948718011379242
92/92 [==============================] - ETA: 0s - loss: 1.5759 - acc: 0.4184
Epoch 00001: val_loss improved from 1.67347 to 1.61844, saving model to BestModCov2.hdf5
204/204 [==============================] - 1s 6ms/step - loss: 1.6184 - acc: 0.3935
HELLO 0.41837432980537415
181/184 [============================>.] - ETA: 0s - loss: 1.5559 - acc: 0.4259
Epoch 00001: val_loss improved from 1.61844 to 1.60607, saving 

KeyboardInterrupt: 